In [ ]:
import pandas as pd
import pyodbc 
import numpy as np
import time
import requests, json
from pathlib import Path  
from datetime import date


#datasource details removed
# api-endpoint
 

with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    columns = ['BBL','DateOpened','HouseNumber','StreetName','City','Zip','SanitationDistrict','SanitationSection']
    values = ['DateOpened','HouseNumber','StreetName','City','Zip','SanitationDistrict','SanitationSection']
    select = ','.join(columns)

    #get data from 311 complaints and merge with df
    sql1 = "SELECT "+select+" FROM [CompostingDB].[compost].[vw_RepeatMissedCollections]"
    df1 = pd.read_sql(sql1,conn)

    #get old data(case records) from salesforce
    sql2 = "SELECT "+select+" FROM [CompostingDB].[compost].[vw_RepeatMissedCollections_SF]"
    df2 = pd.read_sql(sql2,conn)
    
    #get data from Canvas Outreach missed collection
    sql3 = "SELECT "+select+" FROM [CompostingDB].[compost].[vw_RepeatMissedCollections_CanvasOutreach]"
    df3 = pd.read_sql(sql3,conn)
    
    frames = [df1, df2,df3]
    df = pd.concat(frames)
    #df.fillna('', inplace=True) 

    #BEGIN: fill in blank zips
    #get non NAN zip
    nonNanZipDf = df[~pd.isnull(df['Zip'])] 
    nonNanZipDf.drop_duplicates(
        subset = ['BBL', 'Zip'],
        keep = 'last',inplace=True)
    nonNanZipDfOne = nonNanZipDf.groupby(["BBL","Zip"]).tail(1)
    nonNanZipDfOne2 = nonNanZipDfOne[["BBL","Zip"]] 
    df = df.merge(nonNanZipDfOne2, on='BBL', how='left') 
    df['Zip'] = df['Zip_y'].fillna(df['Zip_x'])
    df.drop(['Zip_x', 'Zip_y'], axis=1,inplace=True)
    #END: fill in blank zips

    df['CombinedAddress'] = df['BBL'].astype(str) +':' + df['HouseNumber'].astype(str) +':' + df['StreetName'] +':' + df['City'] +':' + df['Zip'].astype(str)+':' + df['SanitationDistrict']+':' + df['SanitationSection']
    
    df.sort_values(by=['CombinedAddress', 'DateOpened'], inplace=True, ascending=True)
    df['DateOpened'] = pd.to_datetime(df['DateOpened'])
    df = df.loc[(df['DateOpened'] >= '2021-10-04')]
    df.drop_duplicates(inplace=True) 

    #get interval between Complaint dates
    df['DaysInterval'] = df.groupby(['CombinedAddress'])['DateOpened'].transform(lambda x: x.diff().dt.days).fillna(0) 
    
    newColumns = ['BBL','DateOpened','DaysInterval','HouseNumber','StreetName','City','Zip','SanitationDistrict','SanitationSection','CombinedAddress']
    df = df.reindex(columns=newColumns)
    
    #filter out intervals that are not 0 and 6, i.e. complaints that maybe duplicates
    result = df[(df.DaysInterval.eq(0)) | (df.DaysInterval.ge(6))]
    result2 = result.groupby('CombinedAddress')['CombinedAddress'].count()

    #get BBLs/Complaint count of greater than 3. i.e. complaints over 3 consecutive weeks
    result3 = result2[result2.ge(3)]

    details = df[df['CombinedAddress'].isin(result3.index)]
    details.drop_duplicates(inplace=True)
    details['DaysInterval'] = details.groupby(['CombinedAddress'])['DateOpened'].transform(lambda x: x.diff().dt.days).fillna(0)

    #filter out intervals that are not 0 and 6, i.e. complaints that maybe duplicates
    details = details[(details.DaysInterval.eq(0)) | ( details.DaysInterval.ge(6)) ]

    
    summary = details.groupby(["CombinedAddress"])["CombinedAddress"].count().reset_index(name="ComplaintCount")  
    
    lastRowOfGroup = details.groupby(["BBL","HouseNumber","StreetName","City","Zip","SanitationDistrict","SanitationSection"]).tail(1)  
    lastComplaint = lastRowOfGroup[["CombinedAddress","DateOpened"]]

    
    summary2 = pd.merge(summary, lastComplaint, on='CombinedAddress', how='inner')
   
    summary2.rename(columns={"DateOpened": "LastComplaintDate"},inplace=True)
    summary2[['BBL','HouseNumber','StreetName','City','Zip','SanitationDistrict','SanitationSection']] = summary2.CombinedAddress.str.split(":",expand=True)
    summary2.drop(['CombinedAddress'], axis=1,inplace=True)
    
  
    #lastcomplaintdate to date
    summary2['LastComplaintDate'] = summary2['LastComplaintDate'].dt.date
    details[['BBL','HouseNumber','StreetName','City','Zip','SanitationDistrict','SanitationSection']] = details.CombinedAddress.str.split(":",expand=True)
    details['DateOpened'] = details['DateOpened'].dt.date
    details.drop(['CombinedAddress'], axis=1,inplace=True)

    #remove duplicate rowsdf2 = df.drop_duplicates(subset=["Courses", "Fee"], keep=False)
    summary2.sort_values(by=['LastComplaintDate'], inplace=True, ascending=True)
    summary2.drop_duplicates(subset=['BBL','HouseNumber','StreetName','City','Zip','SanitationDistrict','SanitationSection','ComplaintCount'], keep='last', inplace=True)
    #sort from most complaints to least
    summary2.sort_values(by=['ComplaintCount'], inplace=True, ascending=False)
    #convert selected column to string
    
    summary2['HouseNumber'] = summary2['HouseNumber'].astype(str)
    summary2['StreetName'] = summary2['StreetName'].astype(str)
    summary2['Zip'] = summary2['Zip'].astype(str)

    #Created a New Column RowId using BBL+HouseNumber+StreetName Columns
    summary2['RowId'] = summary2[summary2.columns[2:5]].apply(
        lambda x: ':'.join(x.dropna().astype(str)),
        axis=1
    )

    
    #summary2['CombinedAddressTempA] = summary2['BBL']+':'+summary2['HouseNumber'] +':'+ summary2['StreetName'] => join column A
    #df1['CombinedAddressTempB'] = df1['BBL']+':'+df1['HouseNumber'] +':'+ df1['StreetName'] => join column B
    #df1 has the Segment Id,FirstCrossStreetName, SecondCrossStreetName
    
    #carry over Segment Id,FirstCrossStreetName, SecondCrossStreetName to summary2 tab by joined CombinedAddressTempA & CombinedAddressTempB


    #summary2['Latitude']= np.nan
    #summary2['Longitude']= np.nan
    
    '''for index, row in summary2.iterrows():
        params = {'AddressNo':row['HouseNumber'],'StreetName':row['StreetName'],'ZipCode':row['Zip'],'Key':apiKey} 
        try:
            r = requests.request(method='get', url=url, params = params, proxies=proxies)
            data = r.json()
            recGeo = data['display']
            summary2.at[index,'Latitude']=recGeo['out_latitude']
            summary2.at[index,'Longitude']=recGeo['out_longitude']
        except:
            print("An exception occurred. Allowing geocode for a 3-second break...")
            time.sleep(3) # Sleep for 3 seconds
            print("Getting back to geocoding now...")
    '''       
                


with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password) as conn1:
    columns1 = ['first_cross_st','second_cross_st','X_BBL','X_SEGMENT_ID']
    values1 = ['first_cross_st','second_cross_st','X_BBL','X_SEGMENT_ID']

    #get data from 311 COMPLAINTS DYNAMICS and merge with df4
    sql4 = "SELECT [first_cross_st] ,[second_cross_st] ,[X_BBL] ,[X_SEGMENT_ID] ,[X_BBL]+':'+[site_st_num]+':'+[site_st_name] AS RowId FROM [obi_data].[dbo].[c_311_COMPLAINTS_DYNAMICS] WHERE X_SEGMENT_ID IS NOT NULL AND ( [X_BBL]+':'+[site_st_num]+':'+[site_st_name] ) IS NOT NULL GROUP BY [X_BBL]+':'+[site_st_num]+':'+[site_st_name],[first_cross_st],[second_cross_st],[X_BBL],[X_SEGMENT_ID]"
    df4 = pd.read_sql(sql4,conn1)

    
    Final_df = pd.merge(summary2,df4, on='RowId')
    del Final_df['X_BBL']
    del Final_df['RowId']

    #Final_df.columns = Final_df.columns.str.replace('ComplaintCount', 'Complaint Count')
    #Final_df.columns = Final_df.columns.str.replace('LastComplaintDate', 'Last Complaint Date')
    Final_df.columns = Final_df.columns.str.replace('BBL_x', 'BBL')
    #Final_df.columns = Final_df.columns.str.replace('HouseNumber', 'House Number')
    #Final_df.columns = Final_df.columns.str.replace('StreetName', 'Street Name')
    #Final_df.columns = Final_df.columns.str.replace('SanitationDistrict', 'Sanitation District')
    #Final_df.columns = Final_df.columns.str.replace('SanitationSection', 'Sanitation Section')
    #Final_df.columns = Final_df.columns.str.replace('first_cross_st', 'First Cross Street')
    #Final_df.columns = Final_df.columns.str.replace('second_cross_st', 'Second Cross Street')
    #Final_df.columns = Final_df.columns.str.replace('X_SEGMENT_ID', 'Segment Id')


    #YYmmdd
    
    today = date.today()
    d1 = today.strftime("%Y%m%d%H%M%S")

    writer = pd.ExcelWriter('H:'+d1+'.xlsx', engine='xlsxwriter')

    Final_df.to_excel(writer, sheet_name='Summary', index=False)


    summary.to_excel(writer, sheet_name='Details', index=False)
    writer.save()
    
    